Chronological notes

# VScode setup

### install extensions
- Django Template (bibhasdn)
- sqlite (alexcvzz) 
    - need to install: sudo apt install sqlite
    - will then be able to see sqlite explorer via cmd + shift + p in vscode

### enter any commands (cmd + shift + P)


- to customize vscode
    - settings open json

# Create Structure

## Container

https://stackoverflow.com/questions/41573587/what-is-the-difference-between-venv-pyvenv-pyenv-virtualenv-virtualenvwrappe

https://realpython.com/python-virtual-environments-a-primer/

### 1. create environ

In [13]:
# Python 2:
## virtualenv env

# Python 3
## python3 -m venv env

### 2. activate environ

In [14]:
## source env/bin/activate

note: 
- create env in a folder
- dont cd into directory, so that can create django project outside of env folder later

## Install dependencies

get requirement.txt

In [6]:
## pip freeze > requirements.txt

to remove all packages

In [3]:
##  pip freeze | xargs pip uninstall

## Create project

- . important at the end for manage.py file to be outside folders

In [7]:
## django-admin startproject djcrm .

## Gitignore file

https://github.com/github/gitignore/blob/master/Python.gitignore

- create .gitignore file in env folder
- copy paste gitignore template

## Migrations

In [17]:
# python manage.py makemigrations

<b>NOTE: </b>
- delete migrations first if necessary
- delete db (sqlite) if necessary

In [18]:
# python manage.py migrate

## Run project

In [16]:
# python manage.py runserver

- always reset runserver for template/ html changes to take place

## Create app

In [20]:
# python manage.py startapp leads

## Add app to settings.py

In [21]:
# INSTALLED_APPS = [
#     'django.contrib.admin',
#     'django.contrib.auth',
#     'django.contrib.contenttypes',
#     'django.contrib.sessions',
#     'django.contrib.messages',
#     'django.contrib.staticfiles',
#     'leads'
# ]

## Resources

https://www.youtube.com/watch?v=fOukA4Qh9QA

# Auth

leads/models.py

### AbstractUser

-  agent = models.ForeignKey("Agent", on_delete = models.CASCADE) 
    - "Agent" Class

In [31]:
from django.db import models
from django.contrib.auth.models import AbstractUser

# Create your models here.

class User(AbstractUser):
    pass

class Lead(models.Model):
    first_name = models.CharField(max_length = 20)
    last_name = models.CharField(max_length = 20)
    age = models.IntegerField(default = 0)
    agent = models.ForeignKey("Agent", on_delete = models.CASCADE)

    def __str__(self):
        return f"{self.first_name} {self.last_name}"

class Agent(models.Model):
    user = models.OneToOneField(User, on_delete= models.CASCADE)

    def __str__(self):
        return f"{self.user.email} {self.user.last_name}"

ModuleNotFoundError: No module named 'django'

settings.py

In [ ]:
# add auth_user_model. 
# django auto searches for the User class in lead/models.py

AUTH_USER_MODEL = 'leads.User'

### superuser

In [28]:
# python manage.py superuser

to add user to model

# Shell

In [26]:
## python manage.py shell

to lead to specific agent

In [ ]:
from leads.models import Lead, Agent

agent1 = Agent.objects.get(user__email = 'galenhew@gmail.com')
Lead.objects.create(first_name = 'Joe', last_name = 'Soap', age = 35, agent = agent1)

# output: <Lead: Lead object (1)>

query to check lead added

In [ ]:
from leads.models import Lead
Lead.objects.all()
# output: <QuerySet [<Lead: Joe Soap>]>

output displays as per 
    def __str__(self):
        return f"{self.first_name} {self.last_name}"

# Admin

register models into admin to see in django admin http://127.0.0.1:8000/admin

In [ ]:
from django.contrib import admin

# Register your models here.

from .models import User, Lead, Agent


admin.site.register(User)
admin.site.register(Lead)
admin.site.register(Agent)

# Views

In [35]:
from leads.views import home_page 

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', home_page)
]

ModuleNotFoundError: No module named 'leads'

## Structure

### Structure 1: create directory in app 
leads/tempaltes/leads/home_page.html

In [36]:
# html boiler

leads/views.py

In [ ]:
def home_page(request):
    return render(request, 'leads/home_page.html')

urls.py

### structure 2: create directory outside app
template/second_page.html

settings.py

In [ ]:
TEMPLATES = [
    {
        'DIRS': [BASE_DIR/ "templates"],

leads/views.py

In [ ]:
def home_page(request):
    return render(request, 'second_page.html')

# don't need app name in front of 'second_page.html' as django recognizes templates/second_page.html

## Views with context, template

leads/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse
from .models import Lead

# Create your views here.

def home_page(request):
    leads = Lead.objects.all()
    context ={
        'leads': leads
    }

    return render(request, 'second_page.html', context)

template/second_page.html

In [ ]:
<!doctype html>

<html lang="en">
<head>
  <meta charset="utf-8">

  <title>The HTML5 Herald</title>
  <meta name="description" content="The HTML5 Herald">
  <meta name="author" content="SitePoint">

  <link rel="stylesheet" href="css/styles.css?v=1.0">

</head>

<body>
  <h1>2nd page</h1>
  <p> this is the 2nd page </p>
  {{leads}}

  <ul>
    {% for lead in leads%}
    <li> {{lead.first_name}} {{lead.last_name}} </li>
    {% endfor%}
  </ul>
  <script src="js/scripts.js"></script>
</body>
</html>

# Urls

## Structure Urls in each app

urls.py

In [ ]:
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('leads/',  include('leads.urls', namespace="leads")),
]

leads/urls.py

- django auto looks for urlpatterns

In [ ]:
from django.urls import path
from .views import home_page

# app_name links to namespace in urls.py
app_name = 'leads'

urlpatterns = [
    path('all/', home_page)
]

## views with details

leads/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse
from .models import Lead

# Create your views here.

def lead_list(request):
    leads = Lead.objects.all()
    context ={
        'leads': leads
    }

    return render(request, 'leads/lead_list.html', context)


def lead_detail(request, pk):
    lead= Lead.objects.get(id= pk)
    context ={
        'lead': lead
    }
    return render(request, "leads/lead_detail.html", context)


leads/urls.py

In [41]:
from django.urls import path
from .views import lead_list, lead_detail

app_name = 'leads'

urlpatterns = [
    path('', lead_list),
    path('<pk>/',lead_detail)
]

ModuleNotFoundError: No module named 'django'

leads/templates/leads/lead_detail.html

- get context from lead_detail view

In [ ]:
<a href="/leads"> go back to leads</a>
<hr/>
  <h1>this is the details of {{ lead.first_name}}</h1>
  <p> this person's age: {{lead.age}} </p>

leads/templates/leads/lead.html

- lead.pk link to go into lead details

In [ ]:
  <h1>this is all our leads</h1>
  {% for lead in leads%}
    <div class= 'lead'>
      <a href = "/leads/{{lead.pk}}/">
      {{ lead.first_name }} 
      {{ lead.age}}
      </a>
    </div>
  {% endfor %}

# Forms 

## Manual Form

### Create

leads/forms.py

In [ ]:
from django import forms


class LeadForm(forms.Form):
    first_name = forms.CharField()
    last_name= forms.CharField()
    age = forms.IntegerField(min_value =0 )

leads/templates/leads/lead_create.html
- form in lead_create.html
- csrf tokens needed
- .as_p --> django align fields 

In [ ]:
<a href="/leads"> Create a new lead</a>
<hr/>
  <h1>create a new lead</h1>
  <form method = "post">
    {% csrf_token%}
    {{form.as_p}}
    <button>Submit</button>
  </form>

leads/urls.py

In [ ]:
from django.urls import path
from .views import lead_list, lead_detail, lead_create

app_name = 'leads'

urlpatterns = [
    path('', lead_list),
    path('<int:pk>/',lead_detail),
    path('create/', lead_create)
]

leads/views.py

- use print() to check form output
- if request -> check if POST, if so post data into form via LeadForm(request.POST) , else form = blank as per context{ LeadForm()}
- create Lead objects
- redirect to /leads after creation

In [ ]:
from django.http.response import HttpResponseNotAllowed
from django.shortcuts import render, redirect
from django.http import HttpResponse
from .models import Lead, Agent
from .forms import LeadForm

# Create your views here.

def lead_create(request):

    form = LeadForm()
    if request.method == "POST":
        print('Receiving a post request')
        form = LeadForm(request.POST)
        if form.is_valid():
            print('the form is valid')
            print(form.cleaned_data)
            first_name = form.cleaned_data['first_name']
            last_name = form.cleaned_data['last_name']
            age = form.cleaned_data['age']
            agent = Agent.objects.first()
            Lead.objects.create(
                first_name = first_name,
                last_name = last_name,
                age = age,
                agent = agent
            )
            print('lead has been created')
            return redirect('/leads')

    context={
        'form': LeadForm()
    }

    return render(request, "leads/lead_create.html", context)

### Update

leads/urls.py 

In [ ]:
urlpatterns = [
    path('', lead_list),
    path('<int:pk>/',lead_detail),
    path('<int:pk>/update/',lead_update),
    path('create/', lead_create),
]

leads/forms.py

- Lead.objects.get(id=pk)
- lead.save()

In [ ]:
def lead_update(request, pk):
    lead = Lead.objects.get(id = pk)
    form = LeadForm()
    # form = LeadForm()
    if request.method == "POST":
        print('Receiving a post request')
        form = LeadForm(request.POST)
        if form.is_valid():
            print('the form is valid')
            print(form.cleaned_data)
            first_name = form.cleaned_data['first_name']
            last_name = form.cleaned_data['last_name']
            age = form.cleaned_data['age']
            lead.first_name = first_name
            lead.last_name = last_name
            lead.age = age
            lead.save()
            print('lead has been created')
            return redirect('/leads')

    context={
        'form': form,
        'lead': lead
    }
    return render(request, "leads/lead_update.html", context)


## ModelForm 

### Create

leads/forms.py

In [ ]:
class LeadModelForm(forms.ModelForm):
    class Meta:
        model = Lead
        field = (
            'first_name',
            'last_name',
            'age',
            'agent',
        )

views.py

- form.save() 
    - LeadModelForm class field attributes will autosave, thus no need to manually save cleaned_data, and create Lead.objects

In [ ]:
from .forms import LeadModelForm 

def lead_create(request):
    form = LeadModelForm()
    # form = LeadForm()
    if request.method == "POST":
        form = LeadModelForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect('/leads')

    context={
        'form': LeadModelForm()
    }

    return render(request, "leads/lead_create.html", context)

## Improve Urls name

leads/urls.py

- use name

In [ ]:
from django.urls import path
from .views import lead_list, lead_detail, lead_create, lead_update, lead_delete

app_name = 'leads'

urlpatterns = [
    path('', lead_list, name ='lead-list'),
    path('<int:pk>/',lead_detail, name = 'lead-detail'),
    path('<int:pk>/update/',lead_update, name = 'lead-update'),
    path('<int:pk>/delete/', lead_delete, name = 'lead-delete'),
    path('create/', lead_create, name = 'lead-create'),
]

template/leads/lead_detail.html

- {% url 'namespace:name' name.pk %}
    - rmb that urls.py, the namespace = 'leads', thus leads:lead-list, or namespace:name 

In [ ]:
<a href="{% url 'leads:lead-list' %}"> go back to leads</a>
<hr/>
<hr/>
  <h1>this is the details of {{ lead.first_name}}</h1>
  <p> this person's age: {{lead.age}} </p>
  <p> agent responsible for the lead is: {{ lead.agent}} </p>
  
  <hr/>
  <a href = "{% url 'leads:lead-update'  lead.pk %}"> Update </a>
  <a href = "{% url 'leads:lead-delete' lead.pk %}"> Delete </a>

urls.py

In [ ]:
urlpatterns = [
    path('admin/', admin.site.urls),
    path('leads/',  include('leads.urls', namespace="leads")),
]


## Template {% block %}

templates/base.html

In [ ]:
<!doctype html>

<html lang="en">
<head>
  <meta charset="utf-8">

  <title>The HTML5 Herald</title>
  <meta name="description" content="The HTML5 Herald">
  <meta name="author" content="SitePoint">

  <link rel="stylesheet" href="css/styles.css?v=1.0">
  <style>
    .lead{
      padding-top: 10px;
      padding-bottom: 10px;
      padding-left: 6px;
      padding-right: 6px;
      margin-top: 10px;
      background-color: #f6f6f6;
      width: 100%;
    }
  </style>
</head>

<body>
    {% block content%}
    {% endblock content %}
    {% include 'scripts.html' %}
  <script src="js/scripts.js"></script>
</body>
</html>

leads/templates/leads/lead_list.html

- django auto search for base.html in global template folder

In [ ]:
{% extends "base.html"%}

{% block content%}
  <a href = "{% url 'leads:lead-create' %}" > create a lead</a>

  <h1>these are all our leads</h1>
  {% for lead in leads%}
    <div class= 'lead'>
      <a href = "{% url 'leads:lead-detail' lead.pk %}">
      {{ lead.first_name }} 
      {{ lead.age}}
      </a>
    </div>
  {% endfor %}

{% endblock content%}

## Generic views

leads/views.py

- queryset argument
    - use to filter list of objects to view
    - https://docs.djangoproject.com/en/3.2/topics/class-based-views/generic-display/
- context_object_name arguemnt
    - need to specify so that html uses the name given, else will use default context name 'object_name' 
        - i.e. {% for lead in object_name %} instead of  {% for lead in leads %} where context_object_name = 'leads'

In [ ]:
from django.views import generic


class LeadListView(generic.ListView):
    template_name = 'leads/lead_list.html'
    queryset= Lead.objects.all()
    context_object_name = 'leads'


class LeadDetailView(generic.DetailView):
    template_name = 'leads/lead_detail.html'
    queryset = Lead.objects.all()
    context_object_name = 'lead'


class LeadCreateView(generic.CreateView):
    template_name = 'leads/lead_create.html'
    form_class= LeadModelForm  

    def get_success_url(self):
        return reverse('leads:lead-list')

class LeadUpdateView(generic.UpdateView):
    template_name = 'leads/lead_update.html'
    form_class = LeadModelForm
    queryset = Lead.objects.all()

    def get_success_url(self):
        return reverse('leads:lead-list')


class LeadDeleteView(generic.DeleteView):
    template_name = 'leads/lead_delete.html'
    queryset = Lead.objects.all()

    def get_success_url(self):
        return reverse('leads:lead-list')

leads/urls.py

In [ ]:
from leads.models import Lead
from django.urls import path

from .views import (
    LeadCreateView, LeadDeleteView, LeadUpdateView, lead_list, lead_detail, lead_create, lead_update, lead_delete,
    LeadListView, LeadDetailView,
)

app_name = 'leads'

urlpatterns = [
    path('', LeadListView.as_view(), name = 'lead-list'),
    path('<int:pk>/', LeadDetailView.as_view(), name ='lead-detail'),
    path('create/', LeadCreateView.as_view(), name = 'lead-create'),
    path('<int:pk>/update/', LeadUpdateView.as_view(), name = 'lead-update'),
    path('<int:pk>/delete/', LeadDeleteView.as_view(), name = 'lead-delete'),
]

# Static files

settings.py

In [ ]:
STATIC_URL = '/static/'

STATICFILES_DIRS = [
    BASE_DIR / 'static'
]

STATIC_ROOT = 'static_root'

urls.py

- if settings.debug 
    - required as in production, will host on other cloud services, thus will not reference own server during production mode

In [ ]:
from django.conf import settings
from django.conf.urls.static import static

if settings.DEBUG:
    urlpatterns += static(settings.STATIC_URL, document_root = settings.STATIC_ROOT)

create global static folder containing:
- css folder -> styles.css
- images folder
- js folder 

i.e. able to see files via path now:
http://127.0.0.1:8000/static/js/main.js

to use css in html file

In [ ]:
{% load static %}
    
<head>
    <link href= "{% static 'css/styles.css' %}" rel= 'stylesheet'/>

# send emails

- https://docs.djangoproject.com/en/3.2/topics/email/


leads/views.py

In [ ]:
class LeadCreateView(generic.CreateView):
    template_name = 'leads/lead_create.html'
    form_class= LeadModelForm  

    def get_success_url(self):
        return reverse('leads:lead-list')

    def form_valid(self, form):
        # to do send email
        send_mail(
            subject = 'A lead has been created',
            message ='go to the site to see the new lead',
            from_email= 'test@test.com',
            recipient_list = ['test2@test.com']
        )
        # resume LeadCreateView 
        return super(LeadCreateView, self).form_valid(form)

settings.py


- in production use EMAIL_BACKEND ='django.core.mail.backends.smtp.EmailBackend'

In [ ]:
# will log email details to console for dev
EMAIL_BACKEND ='django.core.mail.backends.console.EmailBackend'

# default login / logout

templates/registration/login.html

- must be in this folder as the LoginView in django.contrib.auth.views has template_name = 'registration/login.html'
- note that there is no default 'registration/login.html' in django.contrib.auth.views.

tempalates/registration/login.html

{% extends 'base.html'%}
{% block content%}

<form method ='post'>
    {%csrf_token%}
    {{form.as_p}}
    <button type = 'submit'> Submit</button>
</form>

{% endblock content%}

urls.py

In [ ]:
from django.contrib.auth.views import LoginView, LogoutView

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', LandingPageView.as_view(), name = 'landing-page'),
    path('leads/',  include('leads.urls', namespace="leads")),
    path('login/', LoginView.as_view(), name='login'),
    path('logout/', LogoutView.as_view(), name = 'logout'),
]

if setup correct, on signing in superuser, will redirect to http://127.0.0.1:8000/accounts/profile/

settings.py

In [ ]:
# change redirect to /leads instead of http://127.0.0.1:8000/accounts/profile/
LOGIN_REDIRECT_URL= '/leads'

templates/navbar.html

In [ ]:
{% if not request.user.is_authenticated %}
    <a> Signup </a>
{% endif %}

### SignupView

- no default signup view
- has default from django.contrib.auth.forms import UserCreationForm, UsernameField

leads/views.py

In [ ]:
from django.views import generic

class SignupView(generic.CreateView):
    template_name = 'registration/signup.html'
    form_class = CustomUserCreationForm

    def get_success_url(self):
        return reverse('landing-page')

forms.py

- can see from django contrib the default User model django uses
- need to create CustomUserCreationForm as auth.User swapped for leads.User
    - we want to point to our own User model instead of using the Django default User model

In [ ]:
from django.contrib.auth.forms import UserCreationForm, UsernameField
from django.contrib.auth import get_user_model


User = get_user_model()

class CustomUserCreationForm(UserCreationForm):
    class Meta:
        model = User
        fields= ("username",)
        field_classes = {'username': UsernameField}

## Permissions mixins

leads/views.py
- add LoginRequiredMixin to all the views

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin

class LeadDetailView(LoginRequiredMixin, generic.DetailView):
    template_name = 'leads/lead_detail.html'
    queryset = Lead.objects.all()
    context_object_name = 'lead'